### Homework 5

#### Подготовка

In [27]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from catboost import CatBoostClassifier

import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [11]:
df = pd.read_csv("churn_data.csv", index_col='RowNumber')
df.head(3)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Маловероятно, что CustomerId и Surname влияют на результат.

In [18]:
df_copy = df.copy().drop(columns=['CustomerId', 'Surname'])
df_copy.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,
1,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Разделим выборку на тестовую и тренировочную.

In [62]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df_copy, df_copy['Exited'], random_state=42)

In [63]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [64]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [65]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standart', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [66]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

#### 1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
#### 4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3

In [73]:
pipeline_logreg = Pipeline([
    ('features',feats),
    ('classifier_logreg', LogisticRegression(random_state=42))
])

pipeline_cbc = Pipeline([
    ('features',feats),
    ('classifier_cbc', CatBoostClassifier(random_state=42, silent=True))
])

pipeline_rfc = Pipeline([
    ('features',feats),
    ('classifier_rfc', RandomForestClassifier(random_state=42))
])

Подберем оптимальные параметры для каждой модели.

In [162]:
from sklearn.model_selection import GridSearchCV

params_logreg={'classifier_logreg__max_iter':[50, 80, 100, 120, 150],
        }

params_cbc={'classifier_cbc__max_depth':[3, 5, 8],
            'classifier_cbc__learning_rate':[0.5, 0.8],
        }

params_rfc={
            'classifier_rfc__min_samples_leaf':[1, 3, 5],
            'classifier_rfc__min_samples_split':[2, 4, 6],
            'classifier_rfc__max_depth':[None],
            'classifier_rfc__n_estimators':[80, 100]
        }

In [163]:
models_list = [('LogisticRegression', pipeline_logreg, params_logreg), 
               ('CatBoostClassifier', pipeline_cbc, params_cbc),
               ('RandomForestClassifier', pipeline_rfc, params_rfc)]

In [164]:
for model in models_list: 
    grid = GridSearchCV(model[1],
                        param_grid=model[2],
                        cv=6,
                        refit=False)

    search = grid.fit(X_train, y_train)
    print(f'{model[0]} best params: {search.best_params_}')

LogisticRegression best params: {'classifier_logreg__max_iter': 50}
CatBoostClassifier best params: {'classifier_cbc__learning_rate': 0.5, 'classifier_cbc__max_depth': 8}
RandomForestClassifier best params: {'classifier_rfc__max_depth': None, 'classifier_rfc__min_samples_leaf': 1, 'classifier_rfc__min_samples_split': 6, 'classifier_rfc__n_estimators': 80}


Переобучаем модели с лучшими параметрами

In [165]:
pipeline_logreg = Pipeline([
    ('features',feats),
    ('classifier_logreg', LogisticRegression(random_state=42, max_iter=50))
])

pipeline_cbc = Pipeline([
    ('features',feats),
    ('classifier_cbc', CatBoostClassifier(random_state=42, max_depth=8, learning_rate=0.5, silent=True))
])

pipeline_rfc = Pipeline([
    ('features',feats),
    ('classifier_rfc', RandomForestClassifier(random_state=42, max_depth=None, n_estimators=80, 
                                              min_samples_leaf=1, min_samples_split=6))
])

In [166]:
pipeline_logreg.fit(X_train, y_train)
pipeline_cbc.fit(X_train, y_train)
pipeline_rfc.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [167]:
preds_list = []
for name, model in [('LogisticRegression', pipeline_logreg), ('CatBoostClassifier', pipeline_cbc), 
                    ('RandomForestClassifier', pipeline_rfc)]:
    preds = model.predict_proba(X_test)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    # модифицированная метрика с повышенной значимостью recall - объяснение ниже
    fscore = (1.25 * precision * recall) / (0.25 * precision + recall + 1e-10)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print(f'{name} results:\nBest Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                            fscore[ix],
                                                                            precision[ix],
                                                                            recall[ix]))
    preds_list.append([name, model.predict_proba(X_test)[:, 1], 
                       {'precision': precision[ix], 'recall': recall[ix], 
                        'fscore': fscore[ix], 'threshold': thresholds[ix]}])

LogisticRegression results:
Best Threshold=0.370404, F-Score=0.509, Precision=0.537, Recall=0.421
CatBoostClassifier results:
Best Threshold=0.917632, F-Score=0.660, Precision=0.833, Recall=0.360
RandomForestClassifier results:
Best Threshold=0.549171, F-Score=0.686, Precision=0.809, Recall=0.427


#### 2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)

Лучшая модель по метрикам - RandomForestClassifier. 

Наиболее подходящую метрику нужно выбирать по результатам анализа экономической эффективности модели в каждом конкретном случае. В нашем задании нам нужно максимизировать TP (частный случай - зарабатываем 2 доллара - 1 доллар на привлечение) и минимизировать FP (теряем 1 доллар на привлечение и, в случае с клиентом - "спящей собакой", дополнительно 2 доллара) и FN (недополучаем 1 доллар (2 доход - 1 на привлечение)). В случае, если доля всех типов клиентов одинакова (по 25%), более предпочтительным видится минимизация FP (1 + 2 * 0,25 = 1.5 против (2-1) = 1 от FN), а значит более эффективная метрика - f-score с небольшим уклоном в precision. При других долях или неизвестных распределениях клиентов, наиболее эффективен f1-score.

#### 3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.
#### 5. (опционально) Еще раз провести оценку экономической эффективности

In [168]:
preds_list[2]

['RandomForestClassifier',
 array([0.03069444, 0.04025794, 0.31718254, ..., 0.0646131 , 0.08845238,
        0.09079861]),
 {'precision': 0.8091603053435115,
  'recall': 0.42655935613682094,
  'fscore': 0.6860841422857206,
  'threshold': 0.549171176046176}]

In [169]:
def econ_eff(cm, marketing_cost, potential_earnings, sleeping_dog_percent=0.25, lost_profit_flag=True):
    """
    cm - confusion matrix.reshape(4,)
    marketing cost - стоимость привлечения клиента
    potential_earnings - потенциальная выручка (без учета стоимости привлечения)
    sleeping_dog_percent - доля клиентов - "спящих собак"
    lost_profit_flag - считать ли упущенную выгоду
    """
    tp, fp, fn, tn = cm[0], cm[1], cm[2], cm[3]
    return ((tp * (potential_earnings - marketing_cost)) - 
            ((fp * (marketing_cost + sleeping_dog_percent * lost_profit_flag * potential_earnings)) + 
             (fn * lost_profit_flag * (potential_earnings - marketing_cost))))

In [170]:
cm = confusion_matrix(y_test, preds_list[2][1]>preds_list[2][2]['threshold']).reshape(4,)
print(f'Экономический эффект с учетом упущенной выгоды: {econ_eff(cm, 1, 2)}\nБез учета: {econ_eff(cm, 1, 2, 0.25, False)}')

Экономический эффект с учетом упущенной выгоды: 1592.0
Без учета: 1903.0


Посчитаем эффект от модели без оптимизации параметров

In [171]:
pipeline_rfc = Pipeline([
    ('features',feats),
    ('classifier_rfc', RandomForestClassifier(random_state=42))
])
pipeline_rfc.fit(X_train, y_train)
preds = pipeline_rfc.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (1.25 * precision * recall) / (0.25 * precision + recall + 1e-10)
# locate the index of the largest f score
ix = np.argmax(fscore)
cm_not_optim = confusion_matrix(y_test, preds>thresholds[ix]).reshape(4,)
print(f'Экономический эффект с учетом упущенной выгоды: {econ_eff(cm_not_optim, 1, 2)}\nБез учета: {econ_eff(cm_not_optim, 1, 2, 0.25, False)}')

Экономический эффект с учетом упущенной выгоды: 1565.0
Без учета: 1867.0


**Модель с подбором параметров (первая) показала себя несколько лучше.**